In [1]:
import numpy as np
import sys
sys.path.append('.')
from step15_16_main import *

In [2]:
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)


32.0
64.0


In [6]:
print(x.generation)
print(a.generation)
print(y.generation)


0
1
3
